In [25]:
import glob
import xarray as xr
import netCDF4
import h5netcdf
import scipy
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [26]:
file_path=r'C:\Users\Geraint Webb\Documents\FsdEmulation\Data'
data_list_in=glob.glob(file_path+'\ds_in*')
data_list_out=glob.glob(file_path+'\ds_out*')
data_list_in.sort()
data_list_out.sort()
print(data_list_in)
print(data_list_out)

['C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_in.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_inBering.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_inBering30day10.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_inBering30day20.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_inBering30day23.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_inBering30day26.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_inSH.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_inSH30day10.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_inSH30day20.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_inSHWeddell30day10.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_inSHWeddell30day20.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Data\\ds_inWeddell.csv', 'C:\\Users\\Geraint Webb\\Documents\\FsdEmulation\\Dat

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Geraint Webb\AppData\Local\Temp\ipykernel_18980\4284052655.py:2: SyntaxWarning: invalid escape sequence '\d'
  data_list_in=glob.glob(file_path+'\ds_in*')
C:\Users\Geraint Webb\AppData\Local\Temp\ipykernel_18980\4284052655.py:3: SyntaxWarning: invalid escape sequence '\d'
  data_list_out=glob.glob(file_path+'\ds_out*')


In [27]:
data_arrays_in = [np.loadtxt(file, delimiter=',') for file in data_list_in]
data_arrays_out = [np.loadtxt(file, delimiter=',') for file in data_list_out]

# Print the shapes of the arrays to verify
for i, array in enumerate(data_arrays_in):
    print(f"Shape of data_arrays_in[{i}]: {array.shape}")

for i, array in enumerate(data_arrays_out):
    print(f"Shape of data_arrays_out[{i}]: {array.shape}")

Shape of data_arrays_in[0]: (45693, 13)
Shape of data_arrays_in[1]: (41227, 13)
Shape of data_arrays_in[2]: (30775, 13)
Shape of data_arrays_in[3]: (56466, 13)
Shape of data_arrays_in[4]: (95697, 13)
Shape of data_arrays_in[5]: (35113, 13)
Shape of data_arrays_in[6]: (44050, 13)
Shape of data_arrays_in[7]: (41869, 13)
Shape of data_arrays_in[8]: (3, 13)
Shape of data_arrays_in[9]: (16273, 13)
Shape of data_arrays_in[10]: (371, 13)
Shape of data_arrays_in[11]: (15356, 13)
Shape of data_arrays_in[12]: (86622, 13)
Shape of data_arrays_in[13]: (71015, 13)
Shape of data_arrays_in[14]: (27149, 13)
Shape of data_arrays_in[15]: (34371, 13)
Shape of data_arrays_out[0]: (45693, 12)
Shape of data_arrays_out[1]: (41227, 12)
Shape of data_arrays_out[2]: (30775, 12)
Shape of data_arrays_out[3]: (56466, 12)
Shape of data_arrays_out[4]: (95697, 12)
Shape of data_arrays_out[5]: (35113, 12)
Shape of data_arrays_out[6]: (44050, 12)
Shape of data_arrays_out[7]: (41869, 12)
Shape of data_arrays_out[8]: (3,

In [28]:
# Reshape each array in data_arrays_in to the specified shape and concatenate them
reshaped_data_arrays_in = np.concatenate([array.reshape(-1, 13) for array in data_arrays_in])

# Print the shape of the reshaped array to verify
print(f"Shape of reshaped_data_arrays_in: {reshaped_data_arrays_in.shape}")

Shape of reshaped_data_arrays_in: (642050, 13)


In [29]:
# Reshape each array in data_arrays_out to the specified shape and concatenate them
reshaped_data_arrays_out = np.concatenate([array.reshape(-1, 12) for array in data_arrays_out])

# Print the shape of the reshaped array to verify
print(f"Shape of reshaped_data_arrays_out: {reshaped_data_arrays_out.shape}")

Shape of reshaped_data_arrays_out: (642050, 12)


In [30]:
# Clean data_arrays_in and data_arrays_out
cleaned_data_arrays_in = []
cleaned_data_arrays_out = []

for i in range(reshaped_data_arrays_in.shape[0]):
        if not np.sum(reshaped_data_arrays_out[i])==0:
            cleaned_data_arrays_in.append(reshaped_data_arrays_in[i])
            cleaned_data_arrays_out.append(reshaped_data_arrays_out[i])

# Convert lists back to numpy arrays if needed
cleaned_data_arrays_in = np.array(cleaned_data_arrays_in)
cleaned_data_arrays_out = np.array(cleaned_data_arrays_out)

# Print the shapes of the cleaned arrays to verify
print(f"Shape of cleaned_data_arrays_in: {cleaned_data_arrays_in.shape}")
print(f"Shape of cleaned_data_arrays_out: {cleaned_data_arrays_out.shape}")

Shape of cleaned_data_arrays_in: (614242, 13)
Shape of cleaned_data_arrays_out: (614242, 12)


In [31]:
# Calculate the orders of magnitude difference
def calculate_orders_of_magnitude_difference(predictions, actuals):
    differences = torch.abs(predictions - actuals)
    orders_of_magnitude_diff = torch.log10(differences + 1e-10) - torch.log10(torch.abs(actuals) + 1e-10)
    return torch.mean(torch.abs(orders_of_magnitude_diff))

In [32]:
import torch
print(torch.__version__)  # Check the version of PyTorch
print(torch.cuda.is_available())  # Check if CUDA is available
print(torch.cuda.current_device())  # Check the current CUDA device
print(torch.cuda.get_device_name(0))  # Get the name of the GPU

2.5.0
True
0
NVIDIA GeForce GTX 1080


In [59]:
import torch
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch.optim as optim

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Convert data to PyTorch tensors and move to GPU if available
X_train, X_test, y_train, y_test = train_test_split(cleaned_data_arrays_in, cleaned_data_arrays_out, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Define the neural network architecture
class PhysicsInformedNN(nn.Module):
    def __init__(self):
        super(PhysicsInformedNN, self).__init__()
        self.hidden1 = nn.Linear(X_train.shape[1], 256)
        self.hidden2 = nn.Linear(256, 128)
        self.hidden3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, y_train_tensor.shape[1])
        #self.hidden1 = nn.Linear(X_train.shape[1], 100)
        #self.hidden2 = nn.Linear(100, 100)
        #self.output = nn.Linear(100, y_train_tensor.shape[1])
    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.relu(self.hidden3(x))
        x = self.output(x)
        return x

# Initialize the model, loss function, and optimizer
model = PhysicsInformedNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Custom loss function to enforce the sum of outputs to be zero
def custom_loss(output, target):
    mse_loss = criterion(output, target)

    sign_penalty = 0
    for i in range(output.shape[1]):
        sign_penalty += torch.mean(torch.relu(-output[:, i] * torch.sign(target[:, i])))
    sign_penalty = sign_penalty / output.shape[1]
    
    sign_loss = sign_penalty**2
    sum_constraint = torch.sum(output, dim=1)
    sum_loss = torch.mean(sum_constraint ** 2)
    return [mse_loss, sum_loss, sign_loss]

# Training loop
num_epochs = 4000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = custom_loss(outputs, y_train_tensor)
    loss_sum = loss[0] + loss[1] + 1000 * loss[2]
    loss_sum.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], MSE Loss: {loss[0].item():.6f} Sum loss: {loss[1].item():.6f} Sign Loss: {loss[2].item():.6f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    mse = criterion(predictions, y_test_tensor)
    print(f'Test Mean Squared Error: {mse.item():.20f}')

cuda
Epoch [100/4000], MSE Loss: 0.000199 Sum loss: 0.000007 Sign Loss: 0.000000
Epoch [200/4000], MSE Loss: 0.000013 Sum loss: 0.000001 Sign Loss: 0.000000
Epoch [300/4000], MSE Loss: 0.000012 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [400/4000], MSE Loss: 0.000012 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [500/4000], MSE Loss: 0.000011 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [600/4000], MSE Loss: 0.000011 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [700/4000], MSE Loss: 0.000011 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [800/4000], MSE Loss: 0.000011 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [900/4000], MSE Loss: 0.000010 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [1000/4000], MSE Loss: 0.000010 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [1100/4000], MSE Loss: 0.000010 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [1200/4000], MSE Loss: 0.000010 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [1300/4000], MSE Loss: 0.000010 Sum loss: 0.000000 Sign Loss: 0.000000
Epo

In [60]:
ds_in=cleaned_data_arrays_in
ds_out=cleaned_data_arrays_out

In [65]:
k=2500
print(model(torch.tensor(ds_in[k], dtype=torch.float32).to(device)))
print(sum(model(torch.tensor(ds_in[k], dtype=torch.float32).to(device))))
print(ds_in[k])
print(ds_out[k])

tensor([ 1.5209e-04,  1.8293e-04,  2.3211e-04,  2.0906e-04,  2.9681e-04,
         1.8958e-04,  3.4300e-04,  5.5338e-04,  7.8551e-05, -4.4988e-04,
        -2.2279e-04, -1.5576e-03], device='cuda:0', grad_fn=<ViewBackward0>)
tensor(7.2506e-06, device='cuda:0', grad_fn=<AddBackward0>)
[4.89162505e-01 1.25346109e-01 7.97971189e-02 6.17226288e-02
 5.17611504e-02 4.56434675e-02 4.21169475e-02 4.08703759e-02
 4.19333428e-02 2.16227453e-02 2.02643832e-07 2.34035160e-05
 2.08282971e-02]
[ 1.18911266e-05  3.28570604e-05  6.70701265e-05  1.19619071e-04
  1.97485089e-04  3.08703631e-04  4.61630523e-04  6.63734972e-04
  9.20038670e-04 -2.75943801e-03 -2.02643832e-07 -2.34035160e-05]


In [62]:
# Move ds_in and ds_out to tensors on the appropriate device
ds_in_tensor = torch.tensor(ds_in, dtype=torch.float32).to(device)
ds_out_tensor = torch.tensor(ds_out, dtype=torch.float32).to(device)

# Get predictions for the entire dataset in one go
predicted_outputs = model(ds_in_tensor).cpu()
actual_outputs = ds_out_tensor.cpu()

# Compare the signs of the predicted and actual outputs
predicted_signs = torch.sign(predicted_outputs)
actual_signs = torch.sign(actual_outputs)

# Count the number of different signs
negative_sign_differences = torch.sum(predicted_signs != actual_signs).item()

print(f'Total number of incorrect different negative signs: {negative_sign_differences} As a %: {100 * negative_sign_differences / (ds_in.shape[0] * ds_out.shape[1]):.2f}')

Total number of incorrect different negative signs: 462846 As a %: 6.28


In [68]:
# Move ds_in and ds_out to tensors on the appropriate device
ds_in_tensor = torch.tensor(ds_in, dtype=torch.float32).to(device)
ds_out_tensor = torch.tensor(ds_out, dtype=torch.float32).to(device)

# Get predictions for the entire dataset in one go
predicted_outputs = model(ds_in_tensor).cpu()
actual_outputs = ds_out_tensor.cpu()

#Calulate the average % difference between the predicted and actual outputs
percent_diff = torch.mean(torch.abs((predicted_outputs-actual_outputs))).item()

print(f'Average difference between predicted and actual outputs: {percent_diff:.10f}')

Average difference between predicted and actual outputs: 0.0007020539


In [69]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    avg_orders_of_magnitude_diff = calculate_orders_of_magnitude_difference(test_predictions, y_test_tensor)
    print(f'Average Orders of Magnitude Difference: {avg_orders_of_magnitude_diff.item():.6f}')

Average Orders of Magnitude Difference: 1.214884


In [70]:
#High sign test
import torch
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch.optim as optim

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Convert data to PyTorch tensors and move to GPU if available
X_train, X_test, y_train, y_test = train_test_split(cleaned_data_arrays_in, cleaned_data_arrays_out, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Define the neural network architecture
class PhysicsInformedNN(nn.Module):
    def __init__(self):
        super(PhysicsInformedNN, self).__init__()
        self.hidden1 = nn.Linear(X_train.shape[1], 256)
        self.hidden2 = nn.Linear(256, 128)
        self.hidden3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, y_train_tensor.shape[1])
        #self.hidden1 = nn.Linear(X_train.shape[1], 100)
        #self.hidden2 = nn.Linear(100, 100)
        #self.output = nn.Linear(100, y_train_tensor.shape[1])
    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.relu(self.hidden3(x))
        x = self.output(x)
        return x

# Initialize the model, loss function, and optimizer
model = PhysicsInformedNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Custom loss function to enforce the sum of outputs to be zero
def custom_loss(output, target):
    mse_loss = criterion(output, target)

    sign_penalty = 0
    for i in range(output.shape[1]):
        sign_penalty += torch.mean(torch.relu(-output[:, i] * torch.sign(target[:, i])))
    sign_penalty = sign_penalty / output.shape[1]
    
    sign_loss = sign_penalty**2
    sum_constraint = torch.sum(output, dim=1)
    sum_loss = torch.mean(sum_constraint ** 2)
    return [mse_loss, sum_loss, sign_loss]

# Training loop
num_epochs = 4000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = custom_loss(outputs, y_train_tensor)
    loss_sum = loss[0] + loss[1] + 1000000000000000 * loss[2]
    loss_sum.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], MSE Loss: {loss[0].item():.6f} Sum loss: {loss[1].item():.6f} Sign Loss: {loss[2].item():.6f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    mse = criterion(predictions, y_test_tensor)
    print(f'Test Mean Squared Error: {mse.item():.20f}')

cuda
Epoch [100/4000], MSE Loss: 0.018804 Sum loss: 0.011167 Sign Loss: 0.000000
Epoch [200/4000], MSE Loss: 0.022369 Sum loss: 0.002595 Sign Loss: 0.000000
Epoch [300/4000], MSE Loss: 0.021428 Sum loss: 0.002465 Sign Loss: 0.000000
Epoch [400/4000], MSE Loss: 0.020403 Sum loss: 0.002231 Sign Loss: 0.000000
Epoch [500/4000], MSE Loss: 0.019231 Sum loss: 0.002095 Sign Loss: 0.000000
Epoch [600/4000], MSE Loss: 0.018040 Sum loss: 0.001963 Sign Loss: 0.000000
Epoch [700/4000], MSE Loss: 0.016866 Sum loss: 0.001840 Sign Loss: 0.000000
Epoch [800/4000], MSE Loss: 0.015718 Sum loss: 0.001719 Sign Loss: 0.000000
Epoch [900/4000], MSE Loss: 0.014617 Sum loss: 0.001616 Sign Loss: 0.000000
Epoch [1000/4000], MSE Loss: 0.013575 Sum loss: 0.001513 Sign Loss: 0.000000
Epoch [1100/4000], MSE Loss: 0.013018 Sum loss: 0.001473 Sign Loss: 0.000000
Epoch [1200/4000], MSE Loss: 0.012466 Sum loss: 0.001428 Sign Loss: 0.000000
Epoch [1300/4000], MSE Loss: 0.011954 Sum loss: 0.001364 Sign Loss: 0.000000
Epo

In [71]:
k=2500
print(model(torch.tensor(ds_in[k], dtype=torch.float32).to(device)))
print(sum(model(torch.tensor(ds_in[k], dtype=torch.float32).to(device))))
print(ds_in[k])
print(ds_out[k])

tensor([ 6.2230e-02,  7.3560e-02,  1.4425e-02,  3.3060e-02,  1.3424e-02,
         1.5285e-01,  3.9333e-03,  1.2096e-03,  1.3103e-04, -3.0655e-04,
        -3.8174e-04, -3.2172e-01], device='cuda:0', grad_fn=<ViewBackward0>)
tensor(0.0324, device='cuda:0', grad_fn=<AddBackward0>)
[4.89162505e-01 1.25346109e-01 7.97971189e-02 6.17226288e-02
 5.17611504e-02 4.56434675e-02 4.21169475e-02 4.08703759e-02
 4.19333428e-02 2.16227453e-02 2.02643832e-07 2.34035160e-05
 2.08282971e-02]
[ 1.18911266e-05  3.28570604e-05  6.70701265e-05  1.19619071e-04
  1.97485089e-04  3.08703631e-04  4.61630523e-04  6.63734972e-04
  9.20038670e-04 -2.75943801e-03 -2.02643832e-07 -2.34035160e-05]


In [72]:
# Move ds_in and ds_out to tensors on the appropriate device
ds_in_tensor = torch.tensor(ds_in, dtype=torch.float32).to(device)
ds_out_tensor = torch.tensor(ds_out, dtype=torch.float32).to(device)

# Get predictions for the entire dataset in one go
predicted_outputs = model(ds_in_tensor).cpu()
actual_outputs = ds_out_tensor.cpu()

# Compare the signs of the predicted and actual outputs
predicted_signs = torch.sign(predicted_outputs)
actual_signs = torch.sign(actual_outputs)

# Count the number of different signs
negative_sign_differences = torch.sum(predicted_signs != actual_signs).item()

print(f'Total number of incorrect different negative signs: {negative_sign_differences} As a %: {100 * negative_sign_differences / (ds_in.shape[0] * ds_out.shape[1]):.2f}')

Total number of incorrect different negative signs: 152141 As a %: 2.06


In [73]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    avg_orders_of_magnitude_diff = calculate_orders_of_magnitude_difference(test_predictions, y_test_tensor)
    print(f'Average Orders of Magnitude Difference: {avg_orders_of_magnitude_diff.item():.6f}')

Average Orders of Magnitude Difference: 2.495350
